In [1]:
from flask import Flask, request, jsonify
import numpy as np
from sklearn.linear_model import LogisticRegression
from threading import Thread
from IPython.display import display, HTML
import pickle
import warnings
warnings.filterwarnings('ignore')


# Save model (optional)
with open("iso_forest_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("iso_scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open("iso_label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

# Create Flask app
app = Flask(__name__)

@app.route('/')
def home():
    return """
    <h1>ML Model Server</h1>
    <p>Send POST requests to /predict with JSON data: {"features": [value]}</p>
    """



@app.route('/predict', methods=['POST'])
def predict():
    data = request.json

    try:
        # Extract input values
        tx_type = data['type']  # e.g., "TRANSFER"
        amount = float(data['amount'])
        origin_flag = int(data['origin_flag'])
        dest_flag = int(data['dest_flag'])
        hour = int(data.get('hour', 12))  # default to noon if missing
        day = int(data.get('day', 2))     # default to Tuesday
        is_internal = int(origin_flag == 1 and dest_flag == 1)
        is_suspicious_transfer = int(tx_type == "TRANSFER" and origin_flag == 1 and dest_flag == 0)

        # Encode type
        tx_type_encoded = le.transform([tx_type])[0]

        # Final feature array (7 features)
        features = np.array([[tx_type_encoded, amount, origin_flag, dest_flag, hour, day, is_internal]])
        features_scaled = scaler.transform(features)

        # Predict
        pred = model.predict(features_scaled)
        result = int(pred[0] == -1)

        return jsonify({
            "result" : int(pred[0]),
            "anomaly": bool(result),
            "message": "Anomaly detected" if result else "Transaction looks normal"
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 400


def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Start Flask server in background
flask_thread = Thread(target=run_flask)
flask_thread.daemon = True  # Thread will exit when notebook is closed
flask_thread.start()

In [1]:
from flask import Flask, request, jsonify
import numpy as np
from sklearn.linear_model import LogisticRegression
from threading import Thread
from IPython.display import display, HTML
import pickle
import warnings
warnings.filterwarnings('ignore')
import csv
from datetime import datetime
import os
import pandas as pd


with open("logit_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open("label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

# Create Flask app
app = Flask(__name__)

@app.route('/')
def home():
    return """
    <h1>ML Model Server</h1>
    <p>Send POST requests to /predict with JSON data: {"features": [value]}</p>
    """



@app.route('/predict', methods=['POST'])
def predict():
    data = request.json

    try:
        tx_type = data['type']
        amount = float(data['amount'])
        origin_flag = int(data['origin_flag'])
        dest_flag = int(data['dest_flag'])
        hour = int(data.get('hour', 12))
        day = int(data.get('day', 2))
        is_internal = int(origin_flag == 1 and dest_flag == 1)
        is_suspicious_transfer = int(tx_type == "TRANSFER" and origin_flag == 1 and dest_flag == 0)

        tx_type_encoded = le.transform([tx_type])[0]

        features = np.array([[tx_type_encoded, amount, origin_flag, dest_flag,
                              hour, day, is_internal, is_suspicious_transfer]])
        features_scaled = scaler.transform(features)

        pred = model.predict(features_scaled)[0]
        prob = model.predict_proba(features_scaled)[0][1]  # probability of fraud

        explanation = []
        if is_suspicious_transfer:
            explanation.append("TRANSFER from shell to non-shell account.")
        if amount > 10000:
            explanation.append("High amount transaction.")
        if not is_internal:
            explanation.append("External transaction.")

        # Log if fraud
        if pred == 1:
            log_row = [
                datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                tx_type, amount, origin_flag, dest_flag, hour, day, prob
            ]
            file_exists = os.path.isfile("flagged_txns.csv")
            with open("flagged_txns.csv", "a", newline='') as f:
                writer = csv.writer(f)
                if not file_exists:
                    writer.writerow(["timestamp", "type", "amount", "origin_flag", "dest_flag", "hour", "day", "risk_score"])
                writer.writerow(log_row)

        return jsonify({
            "fraud": bool(pred),
            "risk_score": round(prob, 3),
            "explanation": explanation or ["No strong fraud signal detected."]
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 400

@app.route('/dashboard')
def dashboard():
    if not os.path.exists("flagged_txns.csv"):
        return "<h2>No flagged transactions yet.</h2>"

    df = pd.read_csv("flagged_txns.csv")
    latest = df.sort_values(by='risk_score', ascending=False).head(10)

    table_html = latest.to_html(index=False, classes="table table-bordered")

    chart_data = df['risk_score'].round(2).value_counts().sort_index()
    js_data = ','.join(str(v) for v in chart_data.values)
    js_labels = ','.join(str(k) for k in chart_data.index)

    html = f"""
    <html>
    <head>
        <title>Fraud Dashboard</title>
        <style>
            body {{ font-family: Arial; padding: 20px; }}
            .table {{ border-collapse: collapse; width: 100%; margin-top: 20px; }}
            .table td, .table th {{ border: 1px solid #ddd; padding: 8px; }}
            .table th {{ background-color: #f2f2f2; }}
        </style>
    </head>
    <body>
        <h1>🚨 Fraud Risk Dashboard</h1>
        <h3>📌 Top Flagged Transactions</h3>
        {table_html}
        <h3>📊 Risk Score Distribution</h3>
        <canvas id="riskChart" width="500" height="200"></canvas>
        <script>
        const ctx = document.getElementById('riskChart').getContext('2d');
        new Chart(ctx, {{
            type: 'bar',
            data: {{
                labels: [{js_labels}],
                datasets: [{{
                    label: 'Risk Score Count',
                    data: [{js_data}],
                    backgroundColor: 'rgba(255, 99, 132, 0.6)'
                }}]
            }},
            options: {{ scales: {{ y: {{ beginAtZero: true }} }} }}
        }});
        </script>
        <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    </body>
    </html>
    """
    return html


def run_flask():
    app.run(host='0.0.0.0', port=5001, debug=False, use_reloader=False)

# Start Flask server in background
flask_thread = Thread(target=run_flask)
flask_thread.daemon = True  # Thread will exit when notebook is closed
flask_thread.start()

In [2]:
from flask import Flask, request, jsonify
import numpy as np
from sklearn.linear_model import LogisticRegression
from threading import Thread
from IPython.display import display, HTML
import pickle
import warnings
warnings.filterwarnings('ignore')
import csv
from datetime import datetime
import os
import pandas as pd


with open("rf_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open("label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

# Create Flask app
app = Flask(__name__)

@app.route('/')
def home():
    return """
    <h1>ML Model Server</h1>
    <p>Send POST requests to /predict with JSON data: {"features": [value]}</p>
    """



@app.route('/predict', methods=['POST'])
def predict():
    data = request.json

    try:
        tx_type = data['type']
        amount = float(data['amount'])
        origin_flag = int(data['origin_flag'])
        dest_flag = int(data['dest_flag'])
        hour = int(data.get('hour', 12))
        day = int(data.get('day', 2))
        is_internal = int(origin_flag == 1 and dest_flag == 1)
        is_suspicious_transfer = int(tx_type == "TRANSFER" and origin_flag == 1 and dest_flag == 0)

        tx_type_encoded = le.transform([tx_type])[0]

        features = np.array([[tx_type_encoded, amount, origin_flag, dest_flag,
                              hour, day, is_internal, is_suspicious_transfer]])
        features_scaled = scaler.transform(features)

        pred = model.predict(features_scaled)[0]
        prob = model.predict_proba(features_scaled)[0][1]  # probability of fraud

        explanation = []
        if is_suspicious_transfer:
            explanation.append("TRANSFER from shell to non-shell account.")
        if amount > 10000:
            explanation.append("High amount transaction.")
        if not is_internal:
            explanation.append("External transaction.")

        # Log if fraud
        if pred == 1:
            log_row = [
                datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                tx_type, amount, origin_flag, dest_flag, hour, day, prob
            ]
            file_exists = os.path.isfile("flagged_txns.csv")
            with open("flagged_txns.csv", "a", newline='') as f:
                writer = csv.writer(f)
                if not file_exists:
                    writer.writerow(["timestamp", "type", "amount", "origin_flag", "dest_flag", "hour", "day", "risk_score"])
                writer.writerow(log_row)

        return jsonify({
            "fraud": bool(pred),
            "risk_score": round(prob, 3),
            "explanation": explanation or ["No strong fraud signal detected."]
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 400

@app.route('/dashboard')
def dashboard():
    if not os.path.exists("flagged_txns.csv"):
        return "<h2>No flagged transactions yet.</h2>"

    df = pd.read_csv("flagged_txns.csv")
    latest = df.sort_values(by='risk_score', ascending=False).head(10)

    table_html = latest.to_html(index=False, classes="table table-bordered")

    chart_data = df['risk_score'].round(2).value_counts().sort_index()
    js_data = ','.join(str(v) for v in chart_data.values)
    js_labels = ','.join(str(k) for k in chart_data.index)

    html = f"""
    <html>
    <head>
        <title>Fraud Dashboard</title>
        <style>
            body {{ font-family: Arial; padding: 20px; }}
            .table {{ border-collapse: collapse; width: 100%; margin-top: 20px; }}
            .table td, .table th {{ border: 1px solid #ddd; padding: 8px; }}
            .table th {{ background-color: #f2f2f2; }}
        </style>
    </head>
    <body>
        <h1>🚨 Fraud Risk Dashboard</h1>
        <h3>📌 Top Flagged Transactions</h3>
        {table_html}
        <h3>📊 Risk Score Distribution</h3>
        <canvas id="riskChart" width="500" height="200"></canvas>
        <script>
        const ctx = document.getElementById('riskChart').getContext('2d');
        new Chart(ctx, {{
            type: 'bar',
            data: {{
                labels: [{js_labels}],
                datasets: [{{
                    label: 'Risk Score Count',
                    data: [{js_data}],
                    backgroundColor: 'rgba(255, 99, 132, 0.6)'
                }}]
            }},
            options: {{ scales: {{ y: {{ beginAtZero: true }} }} }}
        }});
        </script>
        <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    </body>
    </html>
    """
    return html


def run_flask():
    app.run(host='0.0.0.0', port=5002, debug=False, use_reloader=False)

# Start Flask server in background
flask_thread = Thread(target=run_flask)
flask_thread.daemon = True  # Thread will exit when notebook is closed
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://192.168.0.103:5002


In [ ]:
from flask import Flask, request, jsonify
import numpy as np
from sklearn.linear_model import LogisticRegression
from threading import Thread
from IPython.display import display, HTML
import pickle
import warnings
warnings.filterwarnings('ignore')
import csv
from datetime import datetime
import os
import pandas as pd


with open("rf_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open("label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

# Create Flask app
app = Flask(__name__)

@app.route('/')
def home():
    return """
    <h1>ML Model Server</h1>
    <p>Send POST requests to /predict with JSON data: {"features": [value]}</p>
    """



@app.route('/predict', methods=['POST'])
def predict():
    data = request.json

    try:
        tx_type = data['type']
        amount = float(data['amount'])
        origin_flag = int(data['origin_flag'])
        dest_flag = int(data['dest_flag'])
        hour = int(data.get('hour', 12))
        day = int(data.get('day', 2))
        is_internal = int(origin_flag == 1 and dest_flag == 1)
        is_suspicious_transfer = int(tx_type == "TRANSFER" and origin_flag == 1 and dest_flag == 0)

        tx_type_encoded = le.transform([tx_type])[0]

        features = np.array([[tx_type_encoded, amount, origin_flag, dest_flag,
                              hour, day, is_internal, is_suspicious_transfer]])
        features_scaled = scaler.transform(features)

        pred = model.predict(features_scaled)[0]
        prob = model.predict_proba(features_scaled)[0][1]  # probability of fraud

        explanation = []
        if is_suspicious_transfer:
            explanation.append("TRANSFER from shell to non-shell account.")
        if amount > 10000:
            explanation.append("High amount transaction.")
        if not is_internal:
            explanation.append("External transaction.")

        # Log if fraud
        if pred == 1:
            log_row = [
                datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                tx_type, amount, origin_flag, dest_flag, hour, day, prob
            ]
            file_exists = os.path.isfile("flagged_txns.csv")
            with open("flagged_txns.csv", "a", newline='') as f:
                writer = csv.writer(f)
                if not file_exists:
                    writer.writerow(["timestamp", "type", "amount", "origin_flag", "dest_flag", "hour", "day", "risk_score"])
                writer.writerow(log_row)

        return jsonify({
            "fraud": bool(pred),
            "risk_score": round(prob, 3),
            "explanation": explanation or ["No strong fraud signal detected."]
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 400

In [2]:
from flask import Flask, request, jsonify
import os
from datetime import datetime
import numpy as np
from threading import Thread
from IPython.display import display, HTML
import pickle
import warnings
warnings.filterwarnings('ignore')
import csv
import pandas as pd

app = Flask(__name__)

# Load model
with open("rf_model.pkl", "rb") as f:
    rf_model = pickle.load(f)
with open("iso_model.pkl", "rb") as f:
    iso_model = pickle.load(f)
with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
with open("label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

@app.route('/')
def home():
    return "<h1>Fraud Detection API</h1><p>POST to /predict with transaction data</p>"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json

        tx_type = data['type']
        amount = float(data['amount'])
        origin_flag = int(data['origin_flag'])
        dest_flag = int(data['dest_flag'])
        hour = int(data.get('hour', 12))
        day = int(data.get('day', 2))
        is_internal = int(origin_flag == 1 and dest_flag == 1)
        is_suspicious_transfer = int(tx_type == "TRANSFER" and origin_flag == 1 and dest_flag == 0)


        tx_type_encoded = le.transform([tx_type])[0]

        features = np.array([[tx_type_encoded, amount, origin_flag, dest_flag,
                              hour, day, is_internal, is_suspicious_transfer]])
        features_scaled = scaler.transform(features)

        rf_pred = rf_model.predict(features_scaled)[0]
        rf_prob = rf_model.predict_proba(features_scaled)[0][1]  # risk score

        # Unsupervised anomaly
        iso_pred = iso_model.predict(features_scaled)  # -1 = anomaly
        is_anomaly = int(iso_pred[0] == -1)

        explanation = []

        if amount > 10000:
            explanation.append("High amount transaction.")

        
        is_fraud = int(rf_pred == 1 or is_anomaly == 1)
        
        
        if rf_pred == 1:
            explanation.append("Matched historical fraud patterns (RF)")
        if is_anomaly:
            explanation.append("Unusual pattern detected (IsoForest)")
        if is_suspicious_transfer:
            explanation.append("Suspicious shell transfer")
        if origin_flag == 1 and dest_flag == 1:
            explanation.append("Both origin and destination are shell accounts")
        

        if is_fraud:
            log_row = [datetime.now().strftime("%Y-%m-%d %H:%M:%S"), tx_type, amount,
                       origin_flag, dest_flag, hour, day, rf_prob]
            with open("flagged_txns.csv", "a", newline='') as f:
                writer = csv.writer(f)
                if os.stat("flagged_txns.csv").st_size == 0:
                    writer.writerow(["timestamp", "type", "amount", "origin_flag", "dest_flag", "hour", "day", "risk_score"])
                writer.writerow(log_row)

        return jsonify({
            "fraud": bool(is_fraud),
            "risk_score": round(rf_prob, 3),
            "explanation": explanation or ["Low risk"],
            "iso" : int(iso_pred[0]),
            "i" : is_anomaly
            
        })

        

    except Exception as e:
        return jsonify({"error": str(e)}), 400

@app.route('/dashboard')
def dashboard():
    if not os.path.exists("flagged_txns.csv"):
        return "<h3>No transactions logged yet.</h3>"

    df = pd.read_csv("flagged_txns.csv")
    latest = df.sort_values(by='risk_score', ascending=False).head(10)
    table_html = latest.to_html(index=False, classes="table table-bordered")

    chart_data = df['risk_score'].round(2).value_counts().sort_index()
    js_data = ','.join(str(v) for v in chart_data.values)
    js_labels = ','.join(str(k) for k in chart_data.index)

    html = f"""
    <html><head><title>Dashboard</title>
    <script src='https://cdn.jsdelivr.net/npm/chart.js'></script>
    <style>body {{ font-family: Arial; }} .table {{ border-collapse: collapse; }}
    .table td, .table th {{ border: 1px solid #ddd; padding: 8px; }}</style></head>
    <body>
    <h1>Fraud Risk Dashboard</h1>
    <h3>Top Flagged Transactions</h3>{table_html}
    <h3>Risk Score Histogram</h3>
    <canvas id="riskChart" width="500" height="200"></canvas>
    <script>
    new Chart(document.getElementById('riskChart').getContext('2d'), {{
        type: 'bar',
        data: {{ labels: [{js_labels}], datasets: [{{ label: 'Counts', data: [{js_data}], backgroundColor: 'rgba(255,99,132,0.6)' }}] }},
        options: {{ scales: {{ y: {{ beginAtZero: true }} }} }}
    }});
    </script>
    </body></html>"""
    return html     


def run_flask():
    app.run(host='0.0.0.0', port=5003, debug=False, use_reloader=False)

# Start Flask server in background
flask_thread = Thread(target=run_flask)
flask_thread.daemon = True  # Thread will exit when notebook is closed
flask_thread.start()